<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Evolutionary_Strategies_(ES).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# Define the SimpleModel class
class SimpleModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)  # Fully connected layer

    def forward(self, x):
        return self.fc(x)  # Apply the linear transformation

# Define the evolutionary strategy function
def evolutionary_strategy(model, data, labels, population_size=50, generations=100, sigma=0.1, learning_rate=0.01):
    criterion = nn.MSELoss()  # Loss function
    original_params = torch.nn.utils.parameters_to_vector(model.parameters()).detach()  # Get original parameters

    for _ in range(generations):
        noise = torch.randn(population_size, original_params.size(0)) * sigma  # Generate noise
        rewards = []

        for i in range(population_size):
            new_params = original_params + noise[i]  # Add noise to original parameters
            torch.nn.utils.vector_to_parameters(new_params, model.parameters())  # Set new parameters
            with torch.no_grad():
                output = model(data)  # Forward pass
                loss = criterion(output, labels)  # Compute the loss
            rewards.append(-loss.item())  # Negative loss as reward

        rewards = torch.tensor(rewards)
        normalized_rewards = (rewards - rewards.mean()) / rewards.std()  # Normalize rewards

        gradients = torch.matmul(noise.T, normalized_rewards) / (population_size * sigma)  # Calculate gradients
        updated_params = original_params + learning_rate * gradients  # Update parameters
        torch.nn.utils.vector_to_parameters(updated_params, model.parameters())  # Set updated parameters

    print("Evolutionary strategy optimization completed.")

# Example usage
model = SimpleModel(input_dim=2, output_dim=1)  # Instantiate the model
data = torch.randn(100, 2)  # Generate random data
labels = torch.randn(100, 1)  # Generate random labels
evolutionary_strategy(model, data, labels)  # Perform evolutionary strategy optimization